In [6]:
# data
from torchtext.datasets import WikiText2
from torchtext.data import Field, BucketIterator

# model 
import torch 
import torch.nn as nn
import torch.optim as optim

# training
import tqdm

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
# device

#### Data

In [10]:
BATCH_SIZE = 128

In [51]:
## Fields
text = torchtext.data.Field(
    init_token="<sos>",
    eos_token="<eos",
    lower=True,
    tokenize="spacy",
    tokenizer_language="en",
#     batch_first=True
)

In [52]:
train, val, test = WikiText2.splits(text_field=text)

In [54]:
# build vocabulary
text.build_vocab(train)

In [55]:
# batchification
def batchify(data, bsz):
    data = text.numericalize([data.examples[0].text])
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train, batch_size)
val_data = batchify(val, eval_batch_size)
test_data = batchify(test, eval_batch_size)